In [ ]:
from pygromos.files.topology.mtb import Mtb
from pygromos.files.topology.ifp import ifp

from pygromos.files.blocks import mtb_blocks
from pygromos.data.ff.Gromos54A7 import mtb as mtb_g54a7, ifp as ifp_g54a7

 

## Load Objects

In [ ]:
from pygromos.data.ff.Gromos54A7 import mtb as mtb_g54a7, ifp as ifp_g54a7

mtb_file = Mtb(mtb_g54a7 )
ifp_file = ifp(ifp_g54a7)


In [ ]:
#modify MTB
mtb_file.MTBUILDBLSOLUTE = {v.RNME:v for v in mtb_file.MTBUILDBLSOLUTE_list}
mtb_file.MTBUILDBLSOLVENT = {v.RNMES :v for v in mtb_file.MTBUILDBLSOLVENT_list}
mtb_file.MTBUILDBLEND = {v.RNME:v for v in mtb_file.MTBUILDBLEND_list}

seq = "NH3+ VAL TYR ARG LYSH GLN COO-"
solvent_name = None

In [ ]:
seq = "NH3+ VAL TYR ARG LYSH GLN COO-"
solvent_name = "H2O"

## MakeTop

In [ ]:

residues = seq.split()

#Get Blocks
SOLUTE_bbs = []
bb=None
for i, resn in enumerate(residues):
    if(i==0 or i == len(residues)-1):
        if(resn in mtb_file.MTBUILDBLEND):
            bb = mtb_file.MTBUILDBLEND[resn]
        else:
            raise IOError("The first or last sequence resn was not in the MTBUILDBLEND blocks. Please make sure, it appears there! \n provided: "+str(resn)+"\n possible caps: "+str(list(mtb_file.MTBUILDBLEND.keys())))
    else:
        if(resn in mtb_file.MTBUILDBLSOLUTE):
            bb = mtb_file.MTBUILDBLSOLUTE[resn]
        else:
            raise IOError("The resn <"+str(resn)+"> was not in the MTBUILDBLSOLUTE blocks. Please make sure, it appears there! provided: "+str(resn)+"\n possible caps: "+str(list(mtb_file.MTBUILDBLSOLUTE.keys())))
    SOLUTE_bbs.append(bb)
    
if(not solvent_name is None):
    if(solvent_name in mtb_file.MTBUILDBLSOLVENT):
        solvent_bb = mtb_file.MTBUILDBLSOLVENT[solvent_name]
    else:
        raise IOError("The solvent <"+str(solvent_name)+"> was not in the MTBUILDBLSOLVENT blocks. Please make sure, it appears there! provided: "+str(resn)+"\n possible caps: "+str(list(mtb_file.MTBUILDBLSOLVENT.keys())))
del bb
    

In [ ]:
from pygromos.files.topology import top

#def generate_std_gromos_top
# build topology
# optional TODO: check trailing atoms.
def add_residue(top:top.Top, building_block, preceeding_exclusions):
        atom_offset = len(top.SOLUTEATOM) if(hasattr(top, "SOLUTEATOM")) else 0

        #Add AtomTypes  
        for atom in building_block.atoms:
                top.add_new_atomtype(name=atom.ANM)

        #Add ResN
        if(not isinstance(building_block, mtb_blocks.MTBUILDBLEND)):
                top.add_new_resname(building_block.RNME)
        resi = len(top.RESNAME.content)-1 if(hasattr(top, "RESNAME") and len(top.RESNAME.content)>0) else 1
        
        #Add SoluteBlock
        #TODO: ChargeGroups are arkward
        #TODO: Check INE14 exclusiosn! INE
        #TODO: Masses are rounded, gromos standard is 3 number after digit.
        for i, atom in enumerate(building_block.atoms):
                #lj_params= [i for i in ifp_file.SINGLEATOMLJPAIR if(i.IAC == atom.IACM)][0]
                INE = [atom.MAE]
                INE14 = atom.MSAE
                #print(vars(atom))

                if(len(preceeding_exclusions) > 0 and -i in preceeding_exclusions):
                        INE[0] += preceeding_exclusions[-i].MAE
                        INE14 += preceeding_exclusions[-i].MSAE
                INE14 = list(map(lambda x: x+atom_offset, INE14))
                mass = [i for i in ifp_file.MASSATOMTYPECODE if(i.N == atom.MASS)][0].ATMAS 	 
                #print(vars(atom))
                top.add_new_soluteatom(ATNM=atom.ATOM, MRES=resi, PANM=atom.ANM, IAC=atom.IACM, MASS=mass, 
                                        CG=atom.CGMI, CGC=atom.CGM, INE=atom.MSAE, INE14=atom.MSAE)       
        #1 1 H1 21 1 0 0.248 [2, 3, 4, 5] [2, 3, 4, 5]      
        #Add Bonds
        #TODO: more bonds then expected (expected 61 - got 71 - overlap problem.)
        for bond in building_block.bonds:
                ifp_params = [ifp_bondParams for ifp_bondParams in ifp_file.BONDSTRETCHTYPECODE if(ifp_bondParams.ICB == bond.MCB)][0]
                print(vars(bond), vars(ifp_params))

                includesH = False #ifp_params.atomI.startswith("H") or ifp_params.atomJ.startswith("H")
                top.add_new_bond(atomI=bond.IB+atom_offset, atomJ=bond.JB+atom_offset, 
                                 k=ifp_params.HB, b0=ifp_params.B0, includesH=includesH)
                #I pass on the harmonic bond here... right? - whats with the quarti terms?
        
        #Add Angles
        #TODO: more terms then expected everywhere (expected 61 - got 71 - overlap problem.)
        for angle in bb.angles:
                ifp_params = [i for i in ifp_file.BONDANGLEBENDTYPECODE if(i.ICT == angle.MCB)][0]
                includesH = False #ifp_params.atomI.startswith("H") or ifp_params.atomJ.startswith("H") or ifp_params.atomK.startswith("H")
                top.add_new_angle(k=ifp_params.CT, kh=ifp_params.CHT, b0=ifp_params.T0, 
                                  atomI=angle.IB+atom_offset, atomJ=angle.JB+atom_offset, atomK=angle.KB+atom_offset)
                #I pass on the harmonic bond here... right? - whats with the quarti terms?
         
        #Add Dihedrals     
        #TODO: more terms then expected everywhere (expected 61 - got 71 - overlap problem.)
        for dih in bb.dihedrals:
                ifp_params = [i for i in ifp_file.TORSDIHEDRALTYPECODE if(i.ICP == dih.MCB)][0]
                includesH = False #ifp_params.atomI.startswith("H") or ifp_params.atomJ.startswith("H") or ifp_params.atomK.startswith("H") or ifp_params.atomL.startswith("H")
                top.add_new_torsiondihedral(CP=ifp_params.CP, PD=ifp_params.PD, NP= ifp_params.NP, 
                                                atomI=dih.IB+atom_offset, atomJ=dih.JB+atom_offset, atomK=dih.KB+atom_offset, atomL=dih.LB+atom_offset, includesH=includesH)
        
        #Add Impropers
        #TODO: more terms then expected everywhere (expected 61 - got 71 - overlap problem.)
        for imp in bb.improper_dihedrals:
                ifp_params = [i for i in ifp_file.IMPDIHEDRALTYPECODE if(i.ICQ == imp.MCB)][0]
                includesH = False #ifp_params.atomI.startswith("H") or ifp_params.atomJ.startswith("H") or ifp_params.atomK.startswith("H") or ifp_params.atomL.startswith("H")
                top.add_new_impdihedral(CQ=ifp_params.CQ, Q0=ifp_params.Q0, 
                                        atomI=imp.IB+atom_offset, atomJ=imp.JB+atom_offset, atomK=imp.KB+atom_offset, atomL=imp.LB+atom_offset, includesH=includesH)
   
        #Add LJ-Types
        #for atom in bb.atoms:
        #        top.add_new_LJparameter(C6=, C12=, CS6=, CS12=)


        if(hasattr(bb, "preceding_exclusions")): 
                next_preceeding_exclusions = {i.ATOM:i for i in bb.preceding_exclusions}
        else:
                next_preceeding_exclusions = {}
        return top, next_preceeding_exclusions

def generate_LJ_params(top):
        # IACS mapping.
        IACS = [x.IAC for x in top.SOLUTEATOM]
        if hasattr(top, "SOLVENTATOM"):
                for x in newTop.SOLVENTATOM:
                        IACS.append(x.IACS)
        IACS = {iac:i+1 for i, iac in enumerate(list(set(IACS)))}
        print('IACS\t', len(IACS))

        #Generate LJPARAM Block
        stop_after = len(IACS)/2
        for t,x in enumerate(IACS):
                if(t >= stop_after):
                        break
                for y in IACS:
                        ifp_paramsI = [i for i in ifp_file.SINGLEATOMLJPAIR if(i.IAC == x)][0]
                        ifp_paramsJ = [j for j in ifp_file.SINGLEATOMLJPAIR if(j.IAC == y)][0]
                        
                        C6 = (ifp_paramsI.C6*ifp_paramsJ.C6) #not correct yet
                        C12=ifp_paramsI.C12_1*ifp_paramsJ.C12_1+ifp_paramsI.C12_2*ifp_paramsJ.C12_2+ifp_paramsI.C12_3*ifp_paramsJ.C12_3 #corect
                        
                        CS6 = ifp_paramsI.CS6*ifp_paramsJ.CS6  #corect
                        CS12 = ifp_paramsI.CS12*ifp_paramsJ.CS12  #corect
                        
                        newTop.add_new_LJparameter(C6=C6, C12=C12,
                                                CS6=CS6, CS12=CS12)
                

        #CleanUp IAC numbers
        for atom in newTop.SOLUTEATOM:
                atom.IAC = IACS[atom.IAC]

        for atom in newTop.SOLVENTATOM:
                atom.IACS = IACS[atom.IACS]
        
        return top


In [ ]:
#Translate to TOP
#TODO: consider cyclization
newTop = top.Top(None)
newTop.TITLE = top.blocks.TITLE("A peptide \t seq: "+seq+"\t Force-Field: "+mtb_file.FORCEFIELD.NAME)

newTop.add_block(blocktitle="PHYSICALCONSTANTS",content=[])
newTop.add_block(blocktitle="ATOMTYPENAME", content=[])
newTop.add_block(blocktitle="RESNAME", content=[])


pe = {}
for bb in SOLUTE_bbs:
    newTop, pe = add_residue(newTop, building_block=bb, preceeding_exclusions=pe)

#add  all LJParameters:


solute_atoms = sum([len(bb.atoms) for bb in SOLUTE_bbs])
newTop.add_new_SOLUTEMOLECULES(str(solute_atoms))
newTop.add_new_TEMPERATUREGROUPS(str(solute_atoms))
newTop.add_new_PRESSUREGROUPS(str(solute_atoms))

#SOLVENT?
if(not solvent_bb is None):
    satoms = []
    for solv_atom in solvent_bb.atoms:
        satom = top.blocks.solventatom_type(I=solv_atom.ATOM, ANMS=solv_atom.ANM, IACS=solv_atom.IACM, MASS=solv_atom.MASS, CGS=solv_atom.CG)
        satoms.append(satom)
    newTop.add_block(block=top.blocks.SOLVENTATOM(satoms))

    sconstrs = []
    for solv_const in solvent_bb.constraints:
        sconstr = top.blocks.solventconstr_type(ICONS=solv_const.IB, JCONS=solv_const.JB, CONS=solv_const.LENGTH)
        sconstrs.append(sconstr)
    newTop.add_block(block=top.blocks.SOLVENTCONSTR(sconstrs))

newTop = generate_LJ_params(newTop)



In [ ]:
newTop

In [ ]:
newTop.LJPARAMETERS

In [ ]:
ifp_file.MASSATOMTYPECODE[0].N